# IZV part3: Úkol 2: Test hypotézy
# Autor: Lukáš Kaprál (xkapra00)
Ověřovat s 95% jistotou následující dvě hypotézy
## Hypotéza 1:
**Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na
silnicích třetí třídy**

Import knihoven

In [1]:
import pandas as pd
import scipy as sp

Načteme dataframe s nehodami

In [ ]:
df = pd.read_pickle('accidents.pkl.gz')
df

Vyfiltrujeme pouze nehody na silnicích 1. a 3. třídy

In [ ]:
df = df[df["p36"].isin([1, 3])]
df

Vytvoříme nový sloupec, kam uložíme True/False podle toho, zda při nehodě někdo umřel

In [ ]:
df["died"] = df["p13a"] > 0
df

Tvorba crosstab

In [5]:
ct = pd.crosstab(df["p36"], df["died"])
ct

died,False,True
p36,,
1,78618,911
3,73352,448


Spočítáme 𝜒2

In [6]:
sp.stats.chi2_contingency(ct)[1]

3.5395243450138555e-29

Výsledná hodnota 3.54*10<sup>-29</sup> je menší, než 0.05. Hypotéza tedy neplatí 

## Hypotéza 1:
**Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi**

Vyfiltrujeme pouze vozidla značky Škoda a Audi (p45a 2=AUDI, 39=ŠKODA)

In [7]:
df = df[df["p45a"].isin([2, 39])]
df

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,n,o,p,q,r,s,t,p5a,region,died
10530,003100160575,1,102,2016-06-19,0,1905,1,2,0,2,...,312370,,Souhlasnýsesměremúseku,Pomalý,554782,481548,GN_V0.1UIR-ADR_410,1,PHA,False
50445,002100184560,1,-1,2018-03-20,2,1100,1,2,0,2,...,2775387,,Souhlasnýsesměremúseku,Pomalý,554782,467359,GN_V0.1UIR-ADR_410,1,PHA,False
117991,002100162546,1,0,2016-02-16,2,1110,1,2,0,2,...,1871811,,Souhlasnýsesměremúseku,Pomalý,554782,709450,GN_V0.1UIR-ADR_410,2,STC,False
117994,002100162631,1,0,2016-02-17,3,1010,1,4,0,2,...,,,,,-1,-1,,2,STC,False
119398,010106160002,3,1256,2016-01-04,1,2005,3,0,6,2,...,1013486,,Souhlasnýsesměremúseku,Pomalý,529931,-1,GN_V0.1UIR-ADR_410,2,STC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572838,190906210366,3,2099,2021-07-15,4,1932,3,0,9,2,...,798308,,Souhlasnýsesměremúseku,Pomalý,560537,552526,GN_V0.1UIR-ADR_410,1,KVK,False
572845,190906210373,3,2099,2021-07-19,1,913,1,2,0,2,...,3089393,,Souhlasnýsesměremúseku,Pomalý,560642,-1,GN_V0.1UIR-ADR_410,2,KVK,False
572868,190906210396,3,21036,2021-08-02,1,1605,3,0,4,2,...,1425538,,Souhlasnýsesměremúseku,Pomalý,560588,-1,GN_V0.1UIR-ADR_410,2,KVK,False
572888,190906210416,3,2099,2021-08-02,1,1520,2,0,0,2,...,199919,,Souhlasnýsesměremúseku,Pomalý,560286,553654,GN_V0.1UIR-ADR_410,1,KVK,False


Tvorba crosstab, p53 = škoda na vozidle ve stokorunách

In [8]:
#ct = pd.crosstab(df["p53"], df["p45a"])
#ct
df_meds = df.groupby("p45a").agg({"p53": "median"})
df_meds

,p53
p45a,
2,400.0
39,300.0


Vytvoříme df2 s multiindexem pro rychlejší indexování

In [24]:
df2 = df.set_index(["p45a"])
df2 = df2.loc[:, df2.columns.intersection(['p45a', 'p53'])]
df2


,p53
p45a,
39,500
39,0
39,700
39,500
39,2000
...,...
39,30
39,10
39,150


Spočítáme porovnání dvou konfigurací. Porovnáme mediány a spočítáme výsledek MW U-testu

In [25]:
a = df2.loc[39, "p53"]
b = df2.loc[2, "p53"]

print(a.median(), b.median())
sp.stats.mannwhitneyu(a,b)

300.0 400.0


MannwhitneyuResult(statistic=54545609.5, pvalue=3.0284875285142655e-55)

Výsledek nám říká, že škody na vozidlech značky Škoda a Audi jsou podobné. Přestože je medián hodnot lepší pro značku Škoda, __nemůžeme říct, že by cena nehody na této značce aut byla nižší__.